# Our first Chatbot

Structure of prompt messages to OpenAI

```json
[
    {"role": "system", "content": "SYSTEM PROMPT"},
    {"role": "user", "content": "USER PROMPT"},
    {"role": "assistant", "content": "ASSISTANT RESPONSE"},
    {"role": "user", "content": "NEXT USER PROMPT"}
]
```

A function `chat(message, history)` can tackle this with a list of messages looking like:
```py
[
    ["user said", "assistant replied"],
    ["then user said", "then assistant replied"],
    ...
]
```

In [ ]:
import os, sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    print("OPENAI_API_KEY not loaded.")
    sys.exit(1)

In [ ]:
openai = OpenAI()

def chat(*args):
    messages = []
    if len(args) == 1:
        print("INFO: Gradio is using the modern `messages` format.")
        messages = args[0]
    elif len(args) == 2:
        print("INFO: Gradio is using the deprecated `(message, history)` format.")
        user_message, history = args
        messages = history
        messages.append({"role": "user", "content": user_message})

    system_message = {"role": "system", "content": "You are a helpful AI assistant"}
    messages_with_system = [system_message] + messages

    try:
        stream = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages_with_system,
            stream=True
        )

        full_response = ""
        yield ""

        for chunk in stream:
            content = chunk.choices[0].delta.content or ""
            if content:
                full_response += content
                yield full_response

    except Exception as e:
        yield f"An error occurred: {e}"

In [ ]:
chatbot = gr.Chatbot(
    type = "messages",
    height = 500
)

demo = gr.ChatInterface(
    fn = chat,
    type = "messages",
    chatbot = chatbot,
    title = "AI Chatbot",
    description = "Interract with our new AI chatbot",
    theme = "soft",
    css = "footer{display:none!important}"
)

demo.launch(share=False)